## C: Uncertainty propagation

In [ ]:
import torch
import ase.build
from metatomic.torch.ase_calculator import MetatomicCalculator
from metatomic.torch import ModelOutput

In most cases, the observables of interest from your atomistic simulations are not the direct outputs of the ML model (e.g., energies and forces) for which the LLPR module outputs the uncertainties. It then becomes important to properly _propagate_ the uncertainties to the observables of interest.

We demonstrate an example of uncertainty propagation for the case of defect formation energy in bulk copper. We first compute the defect formation energy:

In [ ]:
# Build a fcc copper structure and replicate it to have more atoms
copper = ase.build.bulk("Cu", "fcc", cubic=True)
copper *= (3, 3, 3)  # 108 atoms

# Create a derived structure where 1 atom has been removed
copper_defective = copper.copy()
del copper_defective[50]  # remove atom number 50

# Set the calculator to both structures
calculator = MetatomicCalculator("pet-mad.pt")
copper.calc = calculator
copper_defective.calc = calculator

# Print the defect formation energy
E_perfect = copper.get_potential_energy()
E_defective = copper_defective.get_potential_energy()
formation_energy = E_defective - (107/108) * E_perfect  # adjust for number of atoms
print("Defect formation energy (eV):", formation_energy)

To perform analytical uncertainty propagation, we need to work directly with the inverse covariance matrix and the last layer features corresponding to the systems:

In [ ]:
# extract inverse covariance and multiplier
inverse_covariance = calculator._model.module.inv_covariance_energy_uncertainty
multiplier = calculator._model.module.multiplier_energy_uncertainty ** 2

# obtain the last layer features for the perfect system
last_layer_features_perfect = calculator.run_model(
    copper,
    outputs={"mtt::aux::energy_last_layer_features": ModelOutput(per_atom=False)},
)["mtt::aux::energy_last_layer_features"].block().values

# obtain the last layer features for the defective system
last_layer_features_defective = calculator.run_model(
    copper_defective,
    outputs={"mtt::aux::energy_last_layer_features": ModelOutput(per_atom=False)},
)["mtt::aux::energy_last_layer_features"].block().values

# compute the feature difference exactly as was done for the energies
feature_difference = last_layer_features_defective - (107/108) * last_layer_features_perfect

# compute the uncertainty
variance = multiplier * torch.einsum("...i,ij,...j->...", feature_difference, inverse_covariance, feature_difference)
uncertainty = variance.item()**0.5
print("Defect formation energy uncertainty (eV):", uncertainty, "[analytical propagation]")

In some cases, analytical uncertainty propagation can be prohibitively complex or expensive. In such cases, one can interact with an ensemble of models, generated at the last-layer of the model following the LLPR covariance, to make an ensemble of predictions and compute the spread of the resulting predictions.

This is demonstrated for the same case below:

In [ ]:
# Find the uncertainty from ensemble propagation, the ensembles have already been
# generated for the loaded model
ensemble_perfect = calculator.run_model(
    copper,
    outputs={"energy": ModelOutput(), "energy_ensemble": ModelOutput(per_atom=False)},
)["energy_ensemble"].block().values

ensemble_defective = calculator.run_model(
    copper_defective,
    outputs={"energy": ModelOutput(), "energy_ensemble": ModelOutput(per_atom=False)},
)["energy_ensemble"].block().values

# Compute the ensemble of defect formation energies
formation_energies_ensemble = ensemble_defective - (107/108) * ensemble_perfect

# Compute the standard deviation across the ensemble predictions
uncertainty_ensemble = formation_energies_ensemble.std().item()
print("Defect formation energy uncertainty (eV):", uncertainty_ensemble, "[ensemble propagation]")

We can see that the two uncertainties are very close (and they would be exactly the same in the limit of an infinite number of ensemble members).